In [ ]:
!pip install transformers torch
!pip install --upgrade transformers bitsandbytes accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = torch.device("cuda")

In [ ]:

from transformers import pipeline

model_name = "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
)

pipe = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    device=0
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
rules = """You are Player {} in a 5 round Prisoner’s Dilemma. This is round {}. The payoff matrix is:
  Both you and opponent cooperate → each gets 3 points
  You cooperate and opponent defects → you get 0; opponent gets 5
  You defect and opponent cooperates → you get 5; opponent gets 0
  Both you and opponent defect → each gets 1 point
Based on your opponent's previous decisions, decide whether to [Cooperate] or [Defect] to ultimately maximize your points..
Please put your final answer, either [Cooperate] or [Defect] within [   ], then terminate.\n"""
numgames = 16
prevgames = ["No previous rounds since it is round 1" for i in range(numgames)]
datasetallprompts = []
datasetanswer = []

def doprompt(playnum, roundnum):
    global numgames
    global datasetallprompts
    global datasetanswer
    temp = [rules.format(0, roundnum) + "\n" + prevgames[i] for i in range(numgames)]
    temp1 = [rules.format(1, roundnum) + "\n" + prevgames[i] for i in range(numgames)]
    temp2 = temp + temp1
    resp = pipe(temp2, batch_size=8,max_new_tokens=550) #[0]['generated_text'][1]['content']
    finout = []
    for i in range(numgames*2):
      cur = resp[i][0]["generated_text"]
      datasetallprompts.append((temp2[i], cur))
      if ("[Cooperate]" in cur) and ("[Defect]" in cur):
          ind1 = cur.rfind("[Cooperate]")
          ind2 = cur.rfind("[Defect]")
          if (ind1 > ind2):
              finout.append("Cooperate")
          else:
              finout.append("Defect")
      elif "[Cooperate]" in cur:
        finout.append("Cooperate")
      elif "[Defect]" in cur:
        finout.append("Defect")
      else:
        finout.append("Error")
    for i in range(numgames*2):
        datasetanswer.append((temp2[i], finout[i]))
    return finout





In [ ]:
doprompt(0, 1)

['Cooperate', 'Cooperate', 'Cooperate', 'Cooperate']

In [ ]:
curscore1 = [0 for i in range(numgames)]
curscore2 = [0 for i in range(numgames)]
update = "Round {}. Player 0 chose to {}, Player 1 chose to {}. Score among all the rounds: Player 0: {}, Player 1: {}."
def prompt(roundnum):
  global curscore1
  global curscore2
  global update
  global prevgames
  nt = doprompt(-1, roundnum)
  a = nt[:numgames]
  b = nt[numgames:]
  for gamenum in range(numgames):
    #print("hi", gamenum, len(a), len(b))
    if a[gamenum] == "Cooperate" and b[gamenum] == "Cooperate":
      curscore1[gamenum] += 3
      curscore2[gamenum] += 3
    elif a[gamenum] == "Cooperate" and b[gamenum] == "Defect":
      curscore1[gamenum] += 0
      curscore2[gamenum] += 5
    elif a[gamenum] == "Defect" and b[gamenum] == "Cooperate":
      curscore1[gamenum] += 5
      curscore2[gamenum] += 0
    else:
      curscore1[gamenum] += 1
      curscore2[gamenum] += 1
    if (prevgames[gamenum] == "No previous rounds since it is round 1"):
      prevgames[gamenum] = "Previous rounds: \n"
    prevgames[gamenum] += update.format(roundnum, a[gamenum], b[gamenum], curscore1[gamenum], curscore2[gamenum]) + "\n"


In [ ]:
prevgames = ["No previous rounds since it is round 1" for i in range(numgames)]
datasetallprompts = []
datasetanswer = []
curscore1 = [0 for i in range(numgames)]
curscore2 = [0 for i in range(numgames)]

for i in range(5): #num rounds
  prompt(i+1)
for i in prevgames:
  print(i)

You are Player 0 in a 5 round Prisoner’s Dilemma. This is round 2. The payoff matrix is:
  Both you and opponent cooperate → each gets 3 points
  You cooperate and opponent defects → you get 0; opponent gets 5
  You defect and opponent cooperates → you get 5; opponent gets 0
  Both you and opponent defect → each gets 1 point
Based on your opponent's previous decisions, decide whether to [Cooperate] or [Defect] to ultimately maximize your points. This is a low level game, so keep your reasoning concise and under 100 words.
Please put your final answer, either [Cooperate] or [Defect] within [   ], then terminate.

Round 1. Player 0 chose 0 cooperate, Player 1 chose to defect. Score among all the arounds: Player 0: 0, Player 1: 5
Round 2. Player 0 chose to cooperate, Player 1 chose to defect. Score among all the arounds: Player 0: 3, Player 1: 5
Round 3. Player 0 chose to defect, Player 1 chose to cooperate. Score among all the arounds: Player 0: 1, Player 1: 3
Round 4. Player 0 chose to 

In [ ]:
import json
with open("datasetallprompts.json", "w") as f:
    json.dump(datasetallprompts, f, indent=2)
with open("datasetanswers.json", "w") as f:
    json.dump(datasetanswer, f, indent=2)